In [ ]:
# 내가 하고싶은 것 -- 특정기간 동안 빈도수에 따른 키워드 추출 (중요도 X 단순빈도 O)

In [1]:
from konlpy.tag import Okt
from collections import Counter
import pandas as pd

In [2]:
data = pd.read_csv(r'C:\Users\jjang\KDT\03 통계기반 NLP 프로젝트\뉴스요약\다음_IT.csv')

In [3]:
df = data.loc[:, ['content', 'writed_at']]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31843 entries, 0 to 31842
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   content    31843 non-null  object
 1   writed_at  31843 non-null  object
dtypes: object(2)
memory usage: 497.7+ KB


In [5]:
df['writed_at'] = pd.to_datetime(df['writed_at'], format='%Y-%m-%d %H:%M:%S')

In [6]:
df_days = df.query('"2023-02-01 00:00:00"<= writed_at <= "2023-02-03 23:59:59"')

In [20]:
# df_days = df_days.drop(['writed_at'], axis='columns')

In [7]:
okt = Okt()
line = []
n_adj = []

for index, row in df_days.iterrows():
    content = row['content']
    line = okt.pos(content)

    for word, tag in line:
        if tag in ["Noun", "Adjective"]:
            n_adj.append(word)

stop_words = "등 수 있다 것 기자 이 를 있는 고 위 및 통해 의 말 위해 전 며 개 더 중 "
stop_words = set(stop_words.split(" "))
n_adj = [word for word in n_adj if not word in stop_words]

remove_one_word = [word for word in n_adj if len(word)>1]
len(remove_one_word)

counts = Counter(n_adj)
keywords = counts.most_common(500)

In [8]:
keywords[:10]

[('기술', 3784),
 ('기업', 3184),
 ('서비스', 3183),
 ('제공', 2521),
 ('전자', 2464),
 ('시장', 2356),
 ('개발', 2229),
 ('사업', 2172),
 ('대표', 2122),
 ('데이터', 1805)]

In [ ]:
# 이 키워드 중에서 어떤 것을 뉴스레터로 제공할 만한 키워드인지 고민해봐야 한다.

In [9]:
name_li = []
for name, num in keywords:
    name_li.append(name)

In [10]:
name_li = name_li[0:10]
name_li

['기술', '기업', '서비스', '제공', '전자', '시장', '개발', '사업', '대표', '데이터']

In [ ]:
# 키워드 10개 --> 뉴스레터 키워드 --> 기사 선정은 어떻게?

In [ ]:
# 방법 1 -- 뉴스 제목 중에서 일치하는 기사 선정 -- 토픽 모델링 해서 토픽 여러개 -- 토픽당 아무 기사 1개 제공

In [11]:
keyword_1 = name_li[0]
keyword_2 = name_li[1]
keyword_3 = name_li[2]

In [12]:
df = pd.read_csv(r'C:\Users\jjang\KDT\03 통계기반 NLP 프로젝트\뉴스요약\다음_IT.csv')

In [13]:
df_keyword_1 = df[df['title'].str.contains(keyword_1)]

In [14]:
df_keyword_1.head()
# '기술'이 제목에 들어있는 기사 df

,platform,main_category,sub_category,title,content,writer,writed_at,url
24,다음,IT,IT기기,"다이슨, 성수동에 헤어테크 기술 체험존 운영",(지디넷코리아=신영빈 기자)글로벌 기술 기업 다이슨코리아가 헤어 제품 팝업 ‘다이슨...,신영빈,2023-02-01 17:50:00,https://v.daum.net/v/20230202151936824
75,다음,IT,IT기기,"엑스플라, 기술 기업 ‘오지스’ 벨리데이터 참여...건강한 블록체인 생태계 확장",오지스. 컴투스홀딩스 컴투스홀딩스 등 컴투스 그룹이 주도하는 글로벌 블록체인 메인넷...,성기훈,2023-02-01 14:01:00,https://v.daum.net/v/20230203100002863
181,다음,IT,IT기기,"LG전자, 캄보디아 학생 자립 지원…에어컨 기술 무료 교육",(지디넷코리아=류은주 기자)LG전자가 에티오피아에 이어 캄보디아서도 젊은 인재들의 ...,류은주,2023-02-02 16:58:00,https://v.daum.net/v/20230212105602176
349,다음,IT,IT기기,"한국로봇산업진흥원, 유망기술 제품화 수행 기업 모집",(지디넷코리아=신영빈 기자)한국로봇산업진흥원은 ‘2023년 유망기술 제품화 지원사업...,신영빈,2023-02-03 15:48:00,https://v.daum.net/v/20230223101604445
365,다음,IT,IT기기,"세계적 물맛 전문가, '코웨이 환경기술연구소' 방문해 호평",(지디넷코리아=이나리 기자)코웨이(대표 서장원)는 물맛 분야의 세계적 권위자인 ‘마...,이나리,2023-02-03 14:05:00,https://v.daum.net/v/20230224134138369


In [19]:
# 제목으로만 토픽 모델링 하여 top 토픽 3개 --> 토픽 1개당 기사 무작위 1개 선정 --> 기사 요약 + URL 제공
# 강사님 실습 파일 vscode로 돌려보기 실패 --> 토픽 모델링은 포기!!

In [ ]:
# 기술 --> 2-gram으로 기술과 함께 언급된 단어 찾기 (title 내에서)